In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [145]:
url = 'https://www.geonames.org/postalcode-search.html?q=mississauga&country=CA&adminCode1=ON'

In [146]:
import requests
results = requests.get(url)
m_geodata = pd.read_html(results.text)

In [147]:
m_geodata

[                                                   0             1
 0  GeoNames Home | Postal Codes | Download / Webs...  search login,
                                                    0   1  \
 0  <!-- google_ad_client = "pub-8752809630410472"... NaN   
 
                                                    2  
 0  PlaceCodeCountryAdmin1Admin2Admin3 1Mississaug...  ,
     Unnamed: 0                                              Place  \
 0          1.0                               Mississauga (Malton)   
 1          NaN                                     43.714/-79.643   
 2          2.0                             Mississauga (Wildwood)   
 3          NaN                                     43.693/-79.607   
 4          3.0                                  Mississauga (YYZ)   
 5          NaN                                      43.69/-79.624   
 6          4.0             Mississauga (Matheson / East Rathwood)   
 7          NaN                                      43.641/-79.63

In [148]:
m_geodata[2]

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Mississauga (Malton),L4T,Canada,Ontario,Mississauga,NaN
1,NaN,43.714/-79.643,43.714/-79.643,43.714/-79.643,43.714/-79.643,43.714/-79.643,43.714/-79.643
2,2.0,Mississauga (Wildwood),L4V,Canada,Ontario,Mississauga,NaN
3,NaN,43.693/-79.607,43.693/-79.607,43.693/-79.607,43.693/-79.607,43.693/-79.607,43.693/-79.607
4,3.0,Mississauga (YYZ),L5P,Canada,Ontario,Mississauga,NaN
5,NaN,43.69/-79.624,43.69/-79.624,43.69/-79.624,43.69/-79.624,43.69/-79.624,43.69/-79.624
6,4.0,Mississauga (Matheson / East Rathwood),L4W,Canada,Ontario,Mississauga,NaN
7,NaN,43.641/-79.63,43.641/-79.63,43.641/-79.63,43.641/-79.63,43.641/-79.63,43.641/-79.63
8,5.0,Mississauga (West Creditview / Mavis / Erindale),L5C,Canada,Ontario,Mississauga,NaN
9,NaN,43.562/-79.65,43.562/-79.65,43.562/-79.65,43.562/-79.65,43.562/-79.65,43.562/-79.65


In [150]:
m_geodata = m_geodata[2].drop(['Country','Admin1', 'Admin2','Admin3', 'Unnamed: 0'], axis=1)
m_geodata = m_geodata.dropna()

In [151]:
m_geodata

,Place,Code
0,Mississauga (Malton),L4T
1,43.714/-79.643,43.714/-79.643
2,Mississauga (Wildwood),L4V
3,43.693/-79.607,43.693/-79.607
4,Mississauga (YYZ),L5P
5,43.69/-79.624,43.69/-79.624
6,Mississauga (Matheson / East Rathwood),L4W
7,43.641/-79.63,43.641/-79.63
8,Mississauga (West Creditview / Mavis / Erindale),L5C
9,43.562/-79.65,43.562/-79.65


In [152]:
m_geodata.columns

Index(['Place', 'Code'], dtype='object')

In [155]:
m_geodata_even = m_geodata.iloc[::2]
m_geodata_odd = m_geodata.iloc[1::2]

In [160]:
m_geodata_even

,Place,Code
0,Mississauga (Malton),L4T
2,Mississauga (Wildwood),L4V
4,Mississauga (YYZ),L5P
6,Mississauga (Matheson / East Rathwood),L4W
8,Mississauga (West Creditview / Mavis / Erindale),L5C
10,Mississauga (Central Lakeview),L5E
12,Mississauga (Clarkson / Southdown),L5J
14,Mississauga (West Sheridan),L5K
16,Mississauga (Lisgar / Meadowvale),L5N
18,Mississauga (West Hurontario / SW Gateway),L5R


In [166]:
for row_pl_co, row_lat_long in zip(m_geodata_even.iterrows(), m_geodata_odd.iterrows()):
    print(row_pl_co[1][0].split('Mississauga (')[1].strip().strip('()'))
#     print(row_pl_co[1][0].split('Mississauga')[1].strip().strip('()'))
#     print([row_pl_co[1][0], row_pl_co[1][1]] + row_lat_long[1][0].split('/'))
#     print(row_pl_co[1][0], row_pl_co[1][1])
#     print(row_lat_long[1][0].split('/'))
    

Malton
Wildwood
YYZ
Matheson / East Rathwood
West Creditview / Mavis / Erindale
Central Lakeview
Clarkson / Southdown
West Sheridan
Lisgar / Meadowvale
West Hurontario / SW Gateway
Cardiff / NE Gateway
Courtney Park / East Gateway
East Credit
Meadowvale Village / West Gateway
East Applewood / East Dixie / NE Lakeview
West Applewood / West Dixie / NW Lakeview
West Rathwood / East Hurontario / SE Gateway
SW Lakeview / Mineola / East Port Credit
Erin Mills / Western Business Park
West Port Credit / Lorne Park / East Sheridan
Churchill Meadows / Central Erin Mills / South Streetsville

West Cooksville / Fairview / City Centre / East Creditview


In [171]:
df = pd.DataFrame(
    [row_pl_co[1][0].split('Mississauga (')[1].strip().strip('()'), row_pl_co[1][1]]+row_lat_long[1][0].split('/') for row_pl_co, row_lat_long in zip(m_geodata_even.iterrows(), m_geodata_odd.iterrows())
)
df.columns = columns=['Neighbourhood','postal_code','Latitude','Longitude']

In [172]:
df

,Neighbourhood,postal_code,Latitude,Longitude
0,Malton,L4T,43.714,-79.643
1,Wildwood,L4V,43.693,-79.607
2,YYZ,L5P,43.69,-79.624
3,Matheson / East Rathwood,L4W,43.641,-79.63
4,West Creditview / Mavis / Erindale,L5C,43.562,-79.65
5,Central Lakeview,L5E,43.584,-79.561
6,Clarkson / Southdown,L5J,43.51,-79.63
7,West Sheridan,L5K,43.527,-79.662
8,Lisgar / Meadowvale,L5N,43.592,-79.761
9,West Hurontario / SW Gateway,L5R,43.606,-79.671


In [173]:
df.shape

(23, 4)

In [174]:
address = 'Mississauga, ON'

geolocator = Nominatim(user_agent="m_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mississauga are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mississauga are 43.590338, -79.645729.


In [182]:
# create map of Mississauga using latitude and longitude values
map_m = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
#for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    #label = '{}'.format(neighborhood)
    #label = folium.Popup(label, parse_html=True)
    #folium.CircleMarker(
        #[lat, lng],
        #radius=5,
        #popup=label,
        #color='blue',
        #fill=True,
        #fill_color='#3186cc',
        #fill_opacity=0.7,
        #parse_html=False).add_to(map_m)  
    
map_m

In [183]:
CLIENT_ID = 'U2GO2U2DJXNDZ1PHXJ3RDECH4JGTLO3CNY2EVN15504F4N4M' # your Foursquare ID
CLIENT_SECRET = 'AH1N0CBESG5HQB3Q2YFDADK3RTUH1H0G5HG1OHV431BIM1KG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U2GO2U2DJXNDZ1PHXJ3RDECH4JGTLO3CNY2EVN15504F4N4M
CLIENT_SECRET:AH1N0CBESG5HQB3Q2YFDADK3RTUH1H0G5HG1OHV431BIM1KG


In [184]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url_1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url_1).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)